In [157]:
import networkx as nx
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---
## 2.4 PageRank

### 2.4.1 Random Surfer Model

#### Exercise 2.12

In [158]:
from random import choice
from collections import defaultdict

In [223]:
from networkx.readwrite import edgelist
G1 = nx.read_adjlist('../data/components.graph', create_using=nx.DiGraph())
G2 = nx.read_adjlist('../data/absorbing.graph', create_using=nx.DiGraph())
W = nx.read_adjlist('../data/wikipedia.graph', create_using=nx.DiGraph())

In [224]:
def random_surfer(G, hops=1000):
    visits = defaultdict(int)
    
    node = choice(G.nodes())
    for _ in range(hops):
        visits[node] += 1
        
        try:
            node = choice(G.neighbors(node))
        except:
            pass
        
    for key in visits :
        print("{}: {}".format(key, visits[key]))

From the components graph, we see that we do not reach the whole network, because there are un-connected components

In [225]:
random_surfer(G1)

4: 284
5: 150
6: 283
7: 283


In the absorbing graph, we get stuck in the node without any out-edges, which therefore gets all the visits.  

In [226]:
random_surfer(G2)

1: 1000


#### Exercise 2.13

In [227]:
from random import random
from collections import defaultdict

In [228]:
DAMPING_FACTOR = 0.15

def page_rank(G, hops=1000):
    visits = defaultdict(int)
    
    node = choice(G.nodes())
    for _ in range(hops):
        visits[node] += 1
        
        if random() > DAMPING_FACTOR:
            neighbors = G.neighbors(node)
            if neighbors:
                node = choice(G.neighbors(node))
            else:
                node = choice(G.nodes())
        else:
            node = choice(G.nodes())
        
    for key in visits :
        print("{}: {}".format(key, visits[key]))
    

In [229]:
page_rank(G1)

0: 173
1: 166
3: 86
2: 173
6: 115
7: 111
4: 116
5: 60


In [230]:
page_rank(G2)

3: 217
0: 168
4: 151
1: 315
2: 149


These results seems to make sence. Nodes with few in-edges get lower score, and if a node has an in-edge only from nodes with many out-edges, it gets 'penalized' for this.

---

### 2.4.2 Power Iteration Method

#### Exercise 2.14: Power Iteration method

In [263]:
titles = pd.DataFrame.from_csv('../data/wikipedia_titles.tsv', sep='\t', )

In [241]:
def page_rank_power(G, hops=1000, delta_lim=1e-3):
    N = len(G.nodes())
    pi = np.array([1/N]*N)
    
    H = np.zeros((N, N))
    for u, v in G.edges_iter():
        o_u = len(G.neighbors(u))
        H[int(u), int(v)] = 1/o_u
        
    dangeling = lambda x: len(G.neighbors(x)) == 0
    w = [1 if dangeling(x) else 0 for x in G.nodes()]
    
    H_hat = H + (w * np.identity(N).T) / N
    
    G = (1 - DAMPING_FACTOR) * H_hat + DAMPING_FACTOR * np.ones((N, N)) / N
    
    for k in range(hops):
        _pi = np.dot(pi, G)
        
        if np.sum(np.abs(pi - _pi)) < delta_lim:
            print("Convergence after {} steps...".format(k))
            break
        if k + 1 == hops:
            print("Reached maximum number of steps: {}...".format(k))
            
        pi = _pi
        
    return pi

In [242]:
wiki_rank = page_rank_power(W)

Convergence after 8 steps...


In [252]:
ranks = np.argsort(wiki_rank)[::-1]

In [268]:
docs = [titles.ix[r]['page_title'] for r in ranks[:10]]

In [274]:
Print top 10 rankings:
for i, title in enumerate(docs):
    print("{}: {}".format(i+1, title))

1: United States
2: United Kingdom
3: France
4: Europe
5: Germany
6: England
7: World War II
8: Latin
9: India
10: English language


---

### 2.4.3 Gaming the system *(Bonus)*

#### Exercise 2.15 *(Bonus)*